# Preparação do ambiente

## Bibliotecas

In [1]:
import base64
import json
import logging
import pandas as pd
import time

from io import BytesIO
from typing import List

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

## Constantes e funções auxiliares

In [ ]:
class PdfGenerator:
    """
     Simple use case:
        pdf_file = PdfGenerator(['https://google.com']).main()
        with open('new_pdf.pdf', "wb") as outfile:
            outfile.write(pdf_file[0].getbuffer())
    """
    driver = None
    # https://chromedevtools.github.io/devtools-protocol/tot/Page#method-printToPDF
    print_options = {
        'landscape': False,
        'displayHeaderFooter': True,
        # 'printBackground': True,
        # 'preferCSSPageSize': True,
        # 'paperWidth': 6.97,
        # 'paperHeight': 16.5,   
    }

    def __init__(self, urls: List[str]):
        self.urls = urls

    def _get_pdf_from_url(self, url, *args, **kwargs):
        self.driver.get(url)
        
        time.sleep(3)  # allow the page to load, increase if needed

        print_options = self.print_options.copy()
        result = self._send_devtools(self.driver, "Page.printToPDF", print_options)
        return base64.b64decode(result['data'])

    @staticmethod
    def _send_devtools(driver, cmd, params):
        """
        Works only with chromedriver.
        Method uses cromedriver's api to pass various commands to it.
        """
        resource = "/session/%s/chromium/send_command_and_get_result" % driver.session_id
        url = driver.command_executor._url + resource
        body = json.dumps({'cmd': cmd, 'params': params})
        response = driver.command_executor._request('POST', url, body)
        return response.get('value')

    def _generate_pdfs(self):
        pdf_files = []

        for url in self.urls:
            result = self._get_pdf_from_url(url)
            file = BytesIO()
            file.write(result)
            pdf_files.append(file)

        return pdf_files

    def main(self) -> List[BytesIO]:
        webdriver_options = ChromeOptions()
        # webdriver_options.add_argument('--headless')
        webdriver_options.add_argument('--disable-gpu')
        webdriver_options.add_argument('--start-maximized')

        try:
            self.driver = webdriver.Chrome(
                service=ChromeService(ChromeDriverManager().install()),
                options=webdriver_options
            )

            # result = self._generate_pdfs()
            result = None
        finally:
            # self.driver.close()
            pass

        return result

In [14]:
print_options = {'landscape': False,  
                 'displayHeaderFooter': True}

webdriver_options = ChromeOptions()
webdriver_options.add_argument('--disable-gpu')
webdriver_options.add_argument('--start-maximized')

driver = webdriver.Chrome(
    service=ChromeService(ChromeDriverManager().install()),
    options=webdriver_options
)

url = 'https://shopee.com.br/'
driver.get(url)


# Carga e prepação dos dados

In [115]:
file = '../datasets/inspecao_ecommerce/carrefour.parquet.gzip'

df = pd.read_parquet(file)
df['Descrição'] = df['Descrição'].apply(lambda x: '' if pd.isna(x) else x)

# columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens', 'Certificado_de_Homologação]
columns_to_keep = ['Palavra_Chave', 'Link', 'Nome']
df = df[columns_to_keep]
df.index = df.index+1
df = df.reset_index()
df.columns = ['Item', 'Texto da Busca', 'Link', 'Descrição do Produto']
df['Arquivo da Página'] = df.apply(lambda row: f'PC202405_Carrefour_{row["Texto da Busca"]}_Item_{row["Item"]}',axis=1)

columns_to_keep = ['Item', 'Texto da Busca', 'Arquivo da Página', 'Link', 'Descrição do Produto']
df = df[columns_to_keep]

df

,Item,Texto da Busca,Arquivo da Página,Link,Descrição do Produto
0,1,bluetooth,PC202405_Carrefour_bluetooth_Item_1,https://www.carrefour.com.br/controle-sem-fio-...,Controle Sem Fio De Xbox Carbon Black Wireless...
1,2,bluetooth,PC202405_Carrefour_bluetooth_Item_2,https://www.carrefour.com.br/headset-gamer-log...,Headset Gamer Logitech G435 Sem Fio Bluetooth ...
2,3,bluetooth,PC202405_Carrefour_bluetooth_Item_3,https://www.carrefour.com.br/fone-de-ouvido-he...,Fone de Ouvido Headset Tectoy Bluetooth XPEAKE...
3,4,bluetooth,PC202405_Carrefour_bluetooth_Item_4,https://www.carrefour.com.br/capa-teclado-blue...,Capa Teclado Bluetooth Anti Impacto Ipad 9 10....
4,5,bluetooth,PC202405_Carrefour_bluetooth_Item_5,https://www.carrefour.com.br/multimidia-automo...,Multimídia Automotivo Pioneer Dmh-a248bt Webli...
...,...,...,...,...,...
2128,2129,wifi,PC202405_Carrefour_wifi_Item_2129,https://www.carrefour.com.br/wifi-network-wire...,Wifi Network Wireless Bt 4.0 Print Server Rede...
2129,2130,wifi,PC202405_Carrefour_wifi_Item_2130,https://www.carrefour.com.br/wifi-network-wire...,Wifi Network Wireless Bt 4.0 Print Server Rede...
2130,2131,wifi,PC202405_Carrefour_wifi_Item_2131,https://www.carrefour.com.br/wifi-wireless-dis...,Wifi Wireless Display Dongle Tv Stick Ultra 4k...
2131,2132,wifi,PC202405_Carrefour_wifi_Item_2132,https://www.carrefour.com.br/wifi-antena-banda...,Wifi Antena Banda Dupla 2.4g / 5.8g (tamanho Ú...


In [137]:
file_pc = r'D:\Users\maxwelfreitas\Downloads\PC_202406_Carrefour_Pasta_de_Análise_Geral 3.XLSX'
df_pc = pd.read_excel(file_pc)
df_pc

,N° do Registro,Nº da Busca,Texto da Busca,Pág.,Item,Arquivo da página (PC_202405_Marketplace_Produto_Pág_item),Data da Análise,Endereço eletrônico (URL) Obs. Usar Copiar Link,Descrição do produto no anúncio,O tipo do produto é de uso proibido no Brasil?,...,Número de Unidades à Venda,Existe o campo código de homologação no anúncio? (Sim ou Não),O código de homologação foi fornecido? (Sim ou Não),"O produto é homologado? (Sim, Não e N.A.)","Validação do código de homologação - O código de homologação fornecido é o do produto anunciado? (Sim, Não, N.A.)",O código EAN foi fornecido? (Sim ou Não) - Apenas para Smartphones.,O código EAN fornecido corresponde ao produto? (Sim ou Não) - Apenas para Smartphones.,Observação 1 - Qual Código de Homologação fornecido no anúncio?,Observação 2 - Qual código de homologação da Anatel para o Produto?,Observação 3
0,5,1,Smartphone,1,5,PC202405_Carrefour_Smartphone_Pág_1_Item_5,2024-06-17,https://www.carrefour.com.br/smartphone-moto-e...,Smartphone Moto EDGE40 NEO 256GB Caneel Bay 5G...,Não,...,1.0,Sim,Sim,Sim,Sim,Não,n.a.,117602300330,117602300330,XT-2307-1
1,35,1,Smartphone,1,35,PC202405_Carrefour_Smartphone_Pág_1_Item_35,2024-06-17,https://www.carrefour.com.br/smartphone-samsun...,Smartphone Samsung Galaxy A35 5G 128GB Azul Es...,Não,...,1.0,Sim,Sim,Sim,Sim,Não,n.a.,235482300953,235482300953,SM-A356E/DS
2,44,1,Smartphone,1,44,PC202405_Carrefour_Smartphone_Pág_1_Item_44,2024-06-17,https://www.carrefour.com.br/smartphone-samsun...,Smartphone Samsung Galaxy S23 Fe 5g 128gb 8gb ...,Não,...,1.0,Sim,Sim,Sim,Não,Não,n.a.,188692101993 (Apple E.T.A. A2589),139732300953,SM-S711B/DS
3,80,1,Smartphone,2,20,PC202405_Carrefour_Smartphone_Pág_2_Item_20,2024-06-17,https://www.carrefour.com.br/smartphone-motoro...,"Smartphone Motorola E22 128GB Preto 4G 6,5"" HD...",Não,...,1.0,Sim,Sim,Sim,Sim,Não,n.a.,126752200330,126752200330,NaN
4,83,1,Smartphone,2,23,PC202405_Carrefour_Smartphone_Pág_2_Item_23,2024-06-17,https://www.carrefour.com.br/smartphone-motoro...,Smartphone Motorola Moto G54 5g Vegan Leather ...,Não,...,1.0,Sim,Sim,Sim,Não,Não,n.a.,188692101993 (Apple E.T.A. A2589),117032300330,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,2130,6,Wi-Fi,6,30,PC202405_Carrefour_Wi-Fi_Pág_6_Item_30,NaT,https://www.carrefour.com.br/lava-e-seca-healt...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,2143,6,Wi-Fi,6,43,PC202405_Carrefour_Wi-Fi_Pág_6_Item_43,NaT,https://www.carrefour.com.br/computador-pc-com...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,2153,6,Wi-Fi,6,53,PC202405_Carrefour_Wi-Fi_Pág_6_Item_53,NaT,https://www.carrefour.com.br/ar-condicionado-t...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,2157,6,Wi-Fi,6,57,PC202405_Carrefour_Wi-Fi_Pág_6_Item_57,NaT,https://www.carrefour.com.br/robo-aspirador-de...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Análise

In [ ]:
# df_pc_to_print = df_pc.iloc[:,[5,7]]
# df_pc_to_print.columns = ['file','url']
# items_to_print = df_pc_to_print.to_dict('records')

# for item in items_to_print:
#     url = item['url']
#     file_name = item['file']
#     file_name = f'carrefour/{file_name}.pdf'
    
    # pdf_file = PdfGenerator([url]).main()
    # with open(file_name, "wb") as outfile:
    #     outfile.write(pdf_file[0].getbuffer())

In [12]:
file_shopee = '../datasets/pdf/shopee/PC_202406_Shopee_Links_WiFI.xlsx'
df_shopee = pd.read_excel(file_shopee)

df_shopee_to_print = df_shopee.iloc[:,[5,7]]
df_shopee_to_print.columns = ['file','url']
items_shopee_to_print = df_shopee_to_print.to_dict('records')

for item in items_shopee_to_print[:5]:
    url = item['url']
    file_name = item['file']
    file_name = f'../datasets/pdf/shopee/{file_name}.pdf'
    
    pdf_file = PdfGenerator([url]).main()
    with open(file_name, "wb") as outfile:
        outfile.write(pdf_file[0].getbuffer())
    print(file_name)

../datasets/pdf/shopee/PC202405_Shopee_Wi-Fi_Pág_1_Item_12.pdf


KeyboardInterrupt: 

In [7]:
item['url'].sp

'https://shopee.com.br/Repetidor-Extensor-Wifi-Amplificador-Sinal-Internet-Wireless-i.415175805.20897628494?sp_atk=243b6231-cbb6-4076-a8e2-90d7bb748ad2&xptdk=243b6231-cbb6-4076-a8e2-90d7bb748ad2'

In [ ]:
url = 'https://shopee.com.br/Lan%C3%A7amento-Celular-Realme-Note-50-4Gb-128Gb-ou-3GB-64Gb-Global-Smartphone-Original-Lacrado-Pronta-Entrega-i.351552985.19799215303?sp_atk=9dc9da9f-f96d-4f5e-b9c4-f2c53e4c9924&xptdk=9dc9da9f-f96d-4f5e-b9c4-f2c53e4c9924'
output_file = '../datasets/pdf/shopee/outputpdf.pdf'
